## Bundled ES modules

While esm.sh is convenient to use, for production use, we recommend creating a standalone bundle. This will load faster and will not require a direct connection to esm.sh, which might not be available in airgapped or firewalled environments.

We will not create a minimal bundle for https://ant.design/

First create a simple file called `antd-minimal.js` that exports what we need.

```javascript
export {Button, Flex, Slider} from 'antd';
```

Next, we install the libraries:

```bash
$ npm install antd
```

And use ESBuild to turn this into a self-contained module/bundle, without react, since ipyreact provides that for us.

```
$ npx esbuild ./antd-minimal.js --bundle --outfile=./antd-minimal.esm.js --format=esm --external:react --external:react-dom --target=esnext
```


Now we can define the module with a custom name (we call it antd-minimal).

In [ ]:
import ipyreact
from pathlib import Path

ipyreact.define_module("antd-minimal", Path("./antd-minimal.esm.js"))

In [ ]:
def on_click(event_data):
    w.children = ["Clicked"]

w = ipyreact.Widget(_module="antd-minimal", _type="Button", children=["Hi there"], events={"onClick": on_click})
w

In [ ]:
stack = ipyreact.Widget(_module="antd-minimal", _type="Flex",
    props={"vertical": True, "style": {"padding": "24px"}},
    children=[
        ipyreact.Widget(_module="antd-minimal", _type="Button", children=["Ant Design Button"]),
        ipyreact.Widget(_module="antd-minimal", _type="Slider",
                       props={"defaultValue": 3, "min": 0, "max": 11}),
])
stack

Input components might need a little bit of custom code, and subclassing `ValueWidget`. It often means binding the value to the right prop of the input component (in this case the Slider takes the same name, `value`) and coupling the event handler (in this case `onChange`) to the `setValue` function.

In [ ]:
import traitlets


class Slider(ipyreact.ValueWidget):
    _esm = """
    
    import {Slider} from "antd-minimal"
    
    export default ({value, setValue, ...rest}) => {
        return <Slider value={value} onChange={(v) => setValue(v)} {...rest}/>
    }
    
    """
s = Slider(value=2)
s

*Note that it depends on the implementation of the event handler if the value is being passed directly, or a (synthetic) event with the data will be passed as argument. An typical example event handler could be `onChange={(event) => setValue(event.target.value)}`.*

Now the slider widget is stateful, and we have bi-directional communication using the `.value` trait.
For instance, we can read it:

In [ ]:
s.value

Or write to it, and it will be reflected directly in the UI.

In [ ]:
s.value = 10